# Sesión práctica de estrategias de *prompting*

Definamos algunas funciones auxiliares que necesitaremos durante el ejercicio

In [4]:
!pip install transformers
!pip freeze ../../requierements.txt


absl-py==2.3.1
accelerate==1.11.0
aiohappyeyeballs==2.6.1
aiohttp==3.13.1
aiosignal==1.4.0
annotated-types==0.7.0
anyio==4.11.0
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
arrow==1.3.0
asttokens==3.0.0
astunparse==1.6.3
async-lru==2.0.5
attrs==25.4.0
babel==2.17.0
beautifulsoup4==4.14.2
bleach==6.2.0
blis==1.3.0
catalogue==2.0.10
certifi==2025.10.5
cffi==2.0.0
chardet==5.2.0
charset-normalizer==3.4.4
click==8.3.0
cloudpathlib==0.23.0
colorama==0.4.6
comm==0.2.3
confection==0.1.5
contourpy==1.3.3
cycler==0.12.1
cymem==2.0.11
datasets==4.3.0
debugpy==1.8.17
decorator==5.2.1
defusedxml==0.7.1
dill==0.4.0
distro==1.9.0
en_core_web_sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl#sha256=1932429db727d4bff3deed6b34cfc05df17794f4a52eeb26cf8928f7c1a0fb85
executing==2.2.1
fastjsonschema==2.21.2
filelock==3.20.0
flatbuffers==25.9.23
fonttools==4.60.1
fqdn==1.5.1
frozenlist==1.8.0
fsspec==2025.9.0
gast==0.6.0
gensim==4

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, set_seed
from transformers.generation.utils import GenerateDecoderOnlyOutput, GenerateBeamDecoderOnlyOutput
import torch

DEVICE = "cpu" # Si no tenéis GPU cambiar a "cpu"
# Si usáis un Google Colab ó un entorno con GPU podéis dejarlo en "cuda:0"
INSTRUCT_MODEL_CON_CASTELLANO = "Qwen/Qwen2.5-0.5B-Instruct" # ~2.4GB en la GPU
model = AutoModelForCausalLM.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(INSTRUCT_MODEL_CON_CASTELLANO)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


def crear_prompt_chat_model(
        mensaje_usuario: str,
        tokenizer: AutoTokenizer,
        historial: str = None,
        mensaje_sistema: str = "You are a helpful assistant."
) -> str:

    if historial is None:
        messages = [{"role": "system", "content": mensaje_sistema}]
    else:
        messages = []

    messages.append({"role": "user", "content": mensaje_usuario})
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    if historial is not None:
        prompt = f"{historial} {prompt}"
    return prompt

def responde_al_mensaje(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        prompt: str,
        streamer: TextStreamer = None,
        gen_kwargs: dict = None,
        return_scores: bool = False,
    ) -> list[str] | tuple[list[str], GenerateDecoderOnlyOutput]:

    gen_kwargs = gen_kwargs or {}
    if return_scores:
        gen_kwargs["return_dict_in_generate"] = True
        gen_kwargs["output_scores"] = True

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    model_output = model.generate(
        input_ids,
        pad_token_id=tokenizer.eos_token_id,
        **gen_kwargs,
        streamer=streamer,
    )
    if isinstance(model_output, GenerateDecoderOnlyOutput):
        # Será el caso si en model.generate hemos pasado return_dict_in_generate=True
        response_tokens = model_output.sequences
    elif isinstance(model_output, torch.Tensor):
        response_tokens = model_output
    elif isinstance(model_output, GenerateBeamDecoderOnlyOutput):
        response_tokens = model_output.sequences
    else:
        raise ValueError(f"El modelo ha devuelto un tipo inesperado: {type(model_output)}")


    responses_txt = tokenizer.batch_decode(response_tokens[:,len(input_ids[0]):], skip_special_tokens=True)

    return responses_txt, model_output

## <a id='toc3_'></a>[Estrategias de *prompting*](#toc0_)

Imaginemos que estamos desarrollando un software para un hospital, cuya función es procesar los emails y whatsapps que llegan solicitando
cita, y extraer una serie de campos de forma estructurada para introducirlos en el sistema de gestión de citas.

En concreto, queremos que el modelo extraiga la siguiente información:

 ```
    {
        "fecha": "<fecha en formato dd/mm/aaaa>",
        "hora": "<hora en formato hh:mm>",
        "especialidad": "<especialidad>",
        "doctor": "<nombre del doctor o doctora>"
    }

```
Indicaciones adicionales:
* Si no se especifica algún campo, se debe incluir un string vacío ""
* Si no especifica la fecha completa (ej. "el viernes"), se debe asumir la fecha del día más próximo que cumpla con sus condiciones
* Si no se especifica la hora:
  * Si se especifica mañana: asignarlo a las 10h
  * Si se especifica tarde: asignarlo a las 18h
  * Si no se especifica o se especifica algo sin sentido (ej. noche), dejarlo vacío

Veamos como podemos usar distintas técnicas de prompting


In [6]:
# Ejemplos de "test" con los que trabajaremos
mail_1 = """Buenas, quería reservar cita a las 10h el 4 de abril con la ginecóloga Macarena García."""
mail_2= """Hola!, tendríais una cita disponible el siguiente viernes por la tarde con algún traumatólogo?"""
output_esperado_1 = {
    "fecha": "04/04/2025",
    "hora": "10:00",
    "especialidad": "ginecología",
    "nombre": "Macarena García"
}
output_esperado_2 = {
    "fecha": "31/01/2025",
    "hora": "18:00",
    "especialidad": "traumatología",
    "doctor": ""
}

In [7]:
# Enfoque más directo

from datetime import datetime, timedelta
import locale
# locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8');

# Set locale NO funciona en Google Colab, lo hacemos "a mano":
dias_semana_str = {
    0: "lunes",
    1: "martes",
    2: "miércoles",
    3: "jueves",
    4: "viernes",
    5: "sábado",
    6: "domingo"
}
meses_str = {
    1: "enero",
    2: "febrero",
    3: "marzo",
    4: "abril",
    5: "mayo",
    6: "junio",
    7: "julio",
    8: "agosto",
    9: "septiembre",
    10: "octubre",
    11: "noviembre",
    12: "diciembre"
}


def crear_prompt_chat_model_citas(mail: str, tokenizer: AutoTokenizer):
    dia_str = dias_semana_str[datetime.today().weekday()]
    mes_str = meses_str[datetime.today().month]
    fecha_hoy_largo = f"{dia_str}, {datetime.today().day} de {mes_str} de {datetime.today().year}"
    msj = f"""
Eres un asistente que gestiona citas en un centro médico. Recibes mails y tu función
es extraer la siguiente información en un formato estructurado tipo:
    {{
        "fecha": "<fecha en formato dd/mm/aaaa>",
        "hora": "<hora en formato hh:mm>",
        "especialidad": "<especialidad>",
        "doctor": "<nombre del doctor o doctora>"
    }}
Indicaciones adicionales:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Hoy es {fecha_hoy_largo}
    - Si no especifica la fecha completa, asume la fecha del día más próximo que cumpla con sus condiciones
    - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
    - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
    - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío

Has recibido este mail:
{mail}
"""
    prompt = crear_prompt_chat_model(msj, tokenizer)
    return prompt

prompt = crear_prompt_chat_model_citas(mail_1, tokenizer)
print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user

Eres un asistente que gestiona citas en un centro médico. Recibes mails y tu función
es extraer la siguiente información en un formato estructurado tipo:
    {
        "fecha": "<fecha en formato dd/mm/aaaa>",
        "hora": "<hora en formato hh:mm>",
        "especialidad": "<especialidad>",
        "doctor": "<nombre del doctor o doctora>"
    }
Indicaciones adicionales:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Hoy es martes, 4 de noviembre de 2025
    - Si no especifica la fecha completa, asume la fecha del día más próximo que cumpla con sus condiciones
    - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
    - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
    - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío

Has recibido este mail:
Buenas, quería reservar cita a las 10h el 4 de

In [8]:
# Veamos el resultado
SEED = 123
set_seed(SEED)
gen_kwargs = {"max_new_tokens": 100, "temperature": 0.5}
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas(mail_1, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)
print("-"*30)
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas(mail_2, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


```json
{
    "fecha": "04/04/2025",
    "hora": "10:00",
    "especialidad": "",
    "doctor": "Macarena García"
}
```

Este formato proporciona una estructura clara para el contenido de la cita, asegurándote de que todos los campos sean completos y útiles.
------------------------------
```json
{
    "fecha": "03/11/2025",
    "hora": "17:00",
    "especialidad": "Traumatología",
    "doctor": "Nombre del Traumatólogo"
}
```


In [9]:
def crear_prompt_chat_model_citas_few_shot(mail: str, tokenizer: AutoTokenizer):

    # Algunos ejemplos necesitan ser dinámicos, ajustados a la fecha de hoy
    año = datetime.today().year
    dia_str = dias_semana_str[datetime.today().weekday()]
    mes_str = meses_str[datetime.today().month]
    fecha_hoy_largo = f"{dia_str}, {datetime.today().day} de {mes_str} de {año}"
    dias_hasta_miércoles = (2 - datetime.today().weekday()) % 7
    if dias_hasta_miércoles == 0:
        dias_hasta_miércoles = 7
    fecha_next_miercoles =  (datetime.today() + timedelta(days=dias_hasta_miércoles)).strftime('%d/%m/%Y')

    _30_este_mes = datetime.today().replace(day=30).strftime('%d/%m/%Y')
    _22_este_mes = datetime.today().replace(day=22)

    if _22_este_mes < datetime.today():
        next_22 = datetime.today().replace(day=22, month=datetime.today().month + 1).strftime('%d/%m/%Y')
    else:
        next_22 = _22_este_mes.strftime('%d/%m/%Y')

    msj = f"""
Eres un asistente que gestiona citas en un centro médico. Recibes mails y tu función
es extraer la siguiente información en un formato estructurado tipo:
    {{
        "fecha": "<fecha en formato dd/mm/aaaa>",
        "hora": "<hora en formato hh:mm>",
        "especialidad": "<especialidad>",
        "doctor": "<nombre del doctor o doctora>"
    }}
Indicaciones adicionales:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Hoy es {fecha_hoy_largo}
    - Si no especifica la fecha completa, asume la fecha del día más próximo que
        cumpla con sus condiciones
    - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
    - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
    - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío

Completa la respuesta para el último mail de este mensaje:

mail: Necesito una cita para el dentista con el Dr. Luis Molina a las 8h del próximo miércoles.
respuesta: {{
    "fecha": "{fecha_next_miercoles}",
    "hora": "08:00",
    "especialidad": "odontología",
    "doctor": "Dr. Luis Molina"
}}

mail: ¿Podrían agendarme con la dermatóloga Clara Ruiz el 30 de este mes a las 3 de la tarde?
respuesta: {{
    "fecha": "{_30_este_mes}",
    "hora": "15:00",
    "especialidad": "dermatólogía",
    "doctor": "Clara Ruiz"
}}

mail: Quiero una cita con cualquier pediatra el 22 por la tarde.
respuesta: {{
    "fecha": "{next_22}",
    "hora": "18:00",
    "especialidad": "pediatría",
    "doctor": ""
}}

mail: Me gustaría agendar una revisión con el cardiólogo el 15 de abril por la mañana. Preferentemente antes del mediodía.
respuesta: {{
    "fecha": "15/04/{año}",
    "hora": "10:00",
    "especialidad": "cardiología",
    "doctor": ""
}}

mail: {mail}
respuesta: """
    prompt = crear_prompt_chat_model(msj, tokenizer)
    return prompt

print(crear_prompt_chat_model_citas_few_shot(mail_1, tokenizer))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user

Eres un asistente que gestiona citas en un centro médico. Recibes mails y tu función
es extraer la siguiente información en un formato estructurado tipo:
    {
        "fecha": "<fecha en formato dd/mm/aaaa>",
        "hora": "<hora en formato hh:mm>",
        "especialidad": "<especialidad>",
        "doctor": "<nombre del doctor o doctora>"
    }
Indicaciones adicionales:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Hoy es martes, 4 de noviembre de 2025
    - Si no especifica la fecha completa, asume la fecha del día más próximo que
        cumpla con sus condiciones
    - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
    - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
    - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío

Completa la respuesta para el último mail de este mensaje:

ma

In [10]:
set_seed(SEED)
gen_kwargs = {"max_new_tokens": 100, "temperature": 0.5}
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas_few_shot(mail_1, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True)
print("-"*30)
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas_few_shot(mail_2, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs, return_scores=True)
# inspeccionar_probabilidades(model_output, response, tokenizer, top_n_probables=5)

{
    "fecha": "04/04/2025",
    "hora": "10:00",
    "especialidad": "ginecología",
    "doctor": "Macarena García"
}
------------------------------
{
    "fecha": "16/11/2025",
    "hora": "17:00",
    "especialidad": "traumatología",
    "doctor": "N/A"
}


In [11]:
# Chain of thought

from datetime import datetime, timedelta
def crear_prompt_chat_model_citas_cot(mail: str, tokenizer):

    # Algunos ejemplos necesitan ser dinámicos, ajustados a la fecha de hoy
    fecha_hoy_corto = datetime.today().strftime('%d/%m/%Y')
    año = datetime.today().year
    dia_str = dias_semana_str[datetime.today().weekday()]
    mes_str = meses_str[datetime.today().month]
    fecha_hoy_largo = f"{dia_str}, {datetime.today().day} de {mes_str} de {año}"
    dias_hasta_miércoles = (2 - datetime.today().weekday()) % 7
    if dias_hasta_miércoles == 0:
        dias_hasta_miércoles = 7
    fecha_next_miercoles =  (datetime.today() + timedelta(days=dias_hasta_miércoles)).strftime('%d/%m/%Y')

    _22_este_mes = datetime.today().replace(day=22)

    if _22_este_mes < datetime.today():
        next_22 = datetime.today().replace(day=22, month=datetime.today().month + 1).strftime('%d/%m/%Y')
    else:
        next_22 = _22_este_mes.strftime('%d/%m/%Y')

    msj = f"""
    Eres un asistente que gestiona citas en un centro médico. Recibes e-mails y tu función
    es extraer la siguiente información en un formato estructurado tipo:
        {{
            "fecha": "<fecha en formato dd/mm/aaaa>",
            "hora": "<hora en formato hh:mm>",
            "especialidad": "<especialidad>",
            "doctor": "<nombre del doctor o doctora>"
        }}
    Indicaciones adicionales:
        - Si no se especifica algún campo, se debe incluir un string vacío ""
        - Hoy es {fecha_hoy_largo}
        - Si no especifica la fecha completa, asume la fecha del día más próximo que
          cumpla con sus condiciones
        - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
        - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
        - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío
        - Antes de dar una respuesta, tienes que pensar paso a paso

    Piensa la respuesta paso a paso para el último mail de este mensaje:

    mail: Necesito una cita para el dentista con el Dr. Luis Molina a las 8h del próximo miércoles.
    piensa la respuesta paso a paso:
        -fecha: dice el próximo miércoles, hoy es {dia_str}, quedan {dias_hasta_miércoles} días hasta el próximo miércoles. Por lo que la fecha será {fecha_hoy_corto} + {dias_hasta_miércoles} días: "{fecha_next_miercoles}"
        -hora: las 8h en formato HH:MM son las "08:00"
        -especialidad: se menciona explícitamente "dentista", un dentista pertence a la rama de la "odontología"
        -doctor: se menciona explícitamente "Dr. Luis Molina"
    por tanto, la respuesta es: {{
    "fecha": "{fecha_next_miercoles}",
    "hora": "08:00",
    "especialidad": "odontología",
    "doctor": "Dr. Luis Molina"
    }}

    mail: Quiero una cita con cualquier pediatra el 22 por la tarde.
    piensa la respuesta paso a paso:
        -fecha: hoy es {fecha_hoy_corto}, el siguiente 22 será el "{next_22}"
        -hora: no menciona una hora, pero menciona "tarde", así que, como dicen las instrucciones, la hora será "18:00"
        -especialidad: se especifica "pediatra", por lo que la especialidad es "pediatría"
        -doctor: no se menciona ningún doctor, por lo que el campo es un string vacío ""
    por tanto, la respuesta es: {{
    "fecha": "{next_22}",
    "hora": "11:30",
    "especialidad": "pediatría",
    "doctor": ""
    }}

    mail: Me gustaría agendar una revisión con el cardiólogo el 15 de abril por la mañana. Preferentemente antes del mediodía.
    piensa la respuesta paso a paso:
        -fecha: la fecha es el 15 de abril, no dice año pero como estamos en {año}, la fecha será "15/04/{año}"
        -hora: pno menciona una hora, pero menciona "mañana", así que, como dicen las instrucciones, la hora será "10:00"
        -especialidad: se especifica "cardiólogo", por lo que la especialidad es "cardiología"
        -doctor: no se menciona ningún doctor, por lo que el campo es un string vacío ""
    por tanto, la respuesta es: {{
    "fecha": "15/04/{año}",
    "hora": "11:00",
    "especialidad": "cardiología",
    "doctor": ""
    }}

    mail: {mail}
    piensa paso a paso:"""
    prompt = crear_prompt_chat_model(msj, tokenizer)
    return prompt

print(crear_prompt_chat_model_citas_cot(mail_1, tokenizer))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user

    Eres un asistente que gestiona citas en un centro médico. Recibes e-mails y tu función
    es extraer la siguiente información en un formato estructurado tipo:
        {
            "fecha": "<fecha en formato dd/mm/aaaa>",
            "hora": "<hora en formato hh:mm>",
            "especialidad": "<especialidad>",
            "doctor": "<nombre del doctor o doctora>"
        }
    Indicaciones adicionales:
        - Si no se especifica algún campo, se debe incluir un string vacío ""
        - Hoy es martes, 4 de noviembre de 2025
        - Si no especifica la fecha completa, asume la fecha del día más próximo que
          cumpla con sus condiciones
        - Si no se especifica la hora pero se especifica "mañana": asignarlo a las 10h
        - Si no se especifica la hora pero se especifica "tarde": asignarlo a las 18h
        - Si no se especifica o se especifica algo sin sentido (ej. noche), déjalo vacío

In [12]:
set_seed(SEED)
gen_kwargs = {"max_new_tokens": 300, "temperature": 0.5}
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas_cot(mail_1, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)
print("-"*30)
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_citas_cot(mail_2, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)

- fecha: el día 4 de abril, ya que es el primer día de abril y no hay otro día disponible para esa hora.
- hora: pno menciona una hora, pero la instrucción dice "mañana", así que, como dicen las instrucciones, la hora será "10:00".
- especialidad: se especifica "ginecóloga", por lo que la especialidad es "ginecología".
- doctor: no se menciona ningún doctor, por lo que el campo es un string vacío ""
por tanto, la respuesta es: {
    "fecha": "04/04/2025",
    "hora": "10:00",
    "especialidad": "ginecología",
    "doctor": ""
}
------------------------------
- Fecha: Hoy es 06/11/2025, el siguiente viernes será el "19/11/2025"
- Horas: No menciona horas específicas, pero según las instrucciones, la hora será "17:00" (siempre es preferible al mediodía)
- Especialidad: Se menciona "traumatólogo", por lo que la especialidad es "tratamiento de lesiones"
- Doctor: Menciona "traumatólogo", por lo que el campo es un string vacío ""
- Respuesta: {
    "fecha": "19/11/2025",
    "hora": "17:00